In [92]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import joblib 
from IPython.display import display
from tqdm import tqdm

In [93]:
from utilities import make_model
class feat_test_mdoel(make_model):
    def __init__(self , name , clf , gamma ,x ,y , train_data_loc):
        self.features = x.columns.to_list() 
        self.train_data_loc = train_data_loc
        make_model.__init__(self , name , clf , gamma , x ,y)

In [94]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['precision' , 'recall' , 'f1_score']
models = ['all_feat_v1' ,'no_color' , 'no_gal_coord' , 'no_IR' , 'no_mw' , 'no_optical_uv']

In [95]:
def score_mean(file):
    all_feat = {
        'AGN' : {
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } ,
        'YSO' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'STAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'HMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'LMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } , 
        'ULX' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'CV':{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'PULSAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        }
    }
    for l in labels:
        for i in range(5):
            model = joblib.load(f'models/{file}_{i}.pkl')
            all_feat[l]['precision'].append(model.result['class_scores']['precision_score'][l])
            all_feat[l]['recall'].append(model.result['class_scores']['recall_score'][l])
            all_feat[l]['f1_score'].append(model.result['class_scores']['f1_score'][l])
    all_mean = {}
    for l in labels:
        all_mean[l] = {}
        for s in scores:
            all_mean[l][s] = np.mean(all_feat[l][s])*100
    all_std = {}
    for l in labels:
        all_std[l] = {}
        for s in scores:
            all_std[l][s] = np.std(all_feat[l][s])*100
    mean_df = pd.DataFrame(all_mean)
    std_df = pd.DataFrame(all_std)
    return mean_df , std_df

In [112]:
score_mean('all_feat_v1')[0].T

,precision,recall,f1_score
AGN,96.844378,97.636743,97.238849
STAR,96.077741,95.698925,95.887958
YSO,92.803856,95.404700,94.086257
HMXB,91.773212,90.668449,91.216149
LMXB,94.769453,80.699301,87.161244
ULX,71.738179,71.184834,71.456748
CV,60.582167,54.457831,57.355057
PULSAR,42.107517,45.346535,43.663625


In [113]:
score_mean('all_feat_v1')[1].T

,precision,recall,f1_score
AGN,0.140244,0.150776,0.108146
STAR,0.078066,0.059976,0.068697
YSO,0.171882,0.254634,0.204430
HMXB,0.236731,0.662541,0.359033
LMXB,1.333332,0.839161,0.598615
ULX,1.226732,1.097236,1.047546
CV,1.161147,1.504819,1.333896
PULSAR,1.930456,1.919873,1.890651


In [106]:
model_all = joblib.load('models/all_feat_v1_1.pkl')
model_all.result['class_scores']['recall_score']

class
AGN       0.977871
CV        0.536145
HMXB      0.914439
LMXB      0.790210
PULSAR    0.455446
STAR      0.957348
ULX       0.710900
YSO       0.953003
Name: recall_score, dtype: float64

In [11]:
final = []
# models = ['all_feat_v1' , 'no_IR' ,  'no_optical_uv' , 'no_mw' ]
models = ['all_feat_v1' , 'no_mw' ]
for mod in tqdm(models):
    mean , std = score_mean(mod)
    for l in labels:
        mean[l] = [f'{m:.1f}$\pm${s:.2f}' for m,s in zip(mean[l] , std[l])]
    final.append(mean)


100%|██████████| 2/2 [00:08<00:00,  4.04s/it]


In [59]:
final[1]

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,91.1$\pm$0.12,89.2$\pm$0.18,82.9$\pm$0.09,92.3$\pm$0.45,94.5$\pm$1.02,61.9$\pm$1.33,56.1$\pm$2.75,28.2$\pm$1.86
recall,95.1$\pm$0.09,88.2$\pm$0.11,89.4$\pm$0.33,90.1$\pm$0.37,81.7$\pm$0.52,43.6$\pm$1.34,45.1$\pm$1.04,18.6$\pm$0.74
f1_score,93.0$\pm$0.07,88.7$\pm$0.14,86.0$\pm$0.15,91.2$\pm$0.39,87.6$\pm$0.55,51.2$\pm$1.36,50.0$\pm$1.69,22.4$\pm$0.87


In [73]:
temp_prec = [pd.concat([final[0].loc['precision'].iloc[i:i+1] , final[1].loc['precision'].iloc[i:i+1]] , keys=['all' , 'no-MW']).to_frame().swaplevel(0,1) for i in range(0,7)]
pd.concat(temp_prec)

precision
AGN  all    96.8$\pm$0.14
     no-MW  91.1$\pm$0.12
STAR all    96.1$\pm$0.08
     no-MW  89.2$\pm$0.18
YSO  all    92.8$\pm$0.17
     no-MW  82.9$\pm$0.09
HMXB all    91.8$\pm$0.24
     no-MW  92.3$\pm$0.45
LMXB all    94.8$\pm$1.33
     no-MW  94.5$\pm$1.02
ULX  all    71.7$\pm$1.23
     no-MW  61.9$\pm$1.33
CV   all    60.6$\pm$1.16
     no-MW  56.1$\pm$2.75

In [75]:
temp_recall = [pd.concat([final[0].loc['recall'].iloc[i:i+1] , final[1].loc['recall'].iloc[i:i+1]] , keys=['all' , 'no-MW']).to_frame().swaplevel(0,1) for i in range(0,7)]
pd.concat(temp_recall)

recall
AGN  all    97.6$\pm$0.15
     no-MW  95.1$\pm$0.09
STAR all    95.7$\pm$0.06
     no-MW  88.2$\pm$0.11
YSO  all    95.4$\pm$0.25
     no-MW  89.4$\pm$0.33
HMXB all    90.7$\pm$0.66
     no-MW  90.1$\pm$0.37
LMXB all    80.7$\pm$0.84
     no-MW  81.7$\pm$0.52
ULX  all    71.2$\pm$1.10
     no-MW  43.6$\pm$1.34
CV   all    54.5$\pm$1.50
     no-MW  45.1$\pm$1.04

In [90]:
score_compact = pd.concat([
    pd.concat([pd.concat([final[0].loc['precision'].iloc[i:i+1] , final[1].loc['precision'].iloc[i:i+1]] , keys=['all-features' , 'no-MW']).to_frame().swaplevel(0,1) for i in range(0,7)]) , 
    pd.concat([pd.concat([final[0].loc['recall'].iloc[i:i+1] , final[1].loc['recall'].iloc[i:i+1]] , keys=['all-features' , 'no-MW']).to_frame().swaplevel(0,1) for i in range(0,7)]),
    pd.concat([pd.concat([final[0].loc['f1_score'].iloc[i:i+1] , final[1].loc['f1_score'].iloc[i:i+1]] , keys=['all-features' , 'no-MW']).to_frame().swaplevel(0,1) for i in range(0,7)])
] , axis=1)
score_compact

precision         recall       f1_score
AGN  all-features  96.8$\pm$0.14  97.6$\pm$0.15  97.2$\pm$0.11
     no-MW         91.1$\pm$0.12  95.1$\pm$0.09  93.0$\pm$0.07
STAR all-features  96.1$\pm$0.08  95.7$\pm$0.06  95.9$\pm$0.07
     no-MW         89.2$\pm$0.18  88.2$\pm$0.11  88.7$\pm$0.14
YSO  all-features  92.8$\pm$0.17  95.4$\pm$0.25  94.1$\pm$0.20
     no-MW         82.9$\pm$0.09  89.4$\pm$0.33  86.0$\pm$0.15
HMXB all-features  91.8$\pm$0.24  90.7$\pm$0.66  91.2$\pm$0.36
     no-MW         92.3$\pm$0.45  90.1$\pm$0.37  91.2$\pm$0.39
LMXB all-features  94.8$\pm$1.33  80.7$\pm$0.84  87.2$\pm$0.60
     no-MW         94.5$\pm$1.02  81.7$\pm$0.52  87.6$\pm$0.55
ULX  all-features  71.7$\pm$1.23  71.2$\pm$1.10  71.5$\pm$1.05
     no-MW         61.9$\pm$1.33  43.6$\pm$1.34  51.2$\pm$1.36
CV   all-features  60.6$\pm$1.16  54.5$\pm$1.50  57.4$\pm$1.33
     no-MW         56.1$\pm$2.75  45.1$\pm$1.04  50.0$\pm$1.69

In [91]:
print(score_compact.to_latex(escape=False))

\begin{tabular}{lllll}
\toprule
   &       &      precision &         recall &       f1_score \\
\midrule
AGN & all-features &  96.8$\pm$0.14 &  97.6$\pm$0.15 &  97.2$\pm$0.11 \\
   & no-MW &  91.1$\pm$0.12 &  95.1$\pm$0.09 &  93.0$\pm$0.07 \\
STAR & all-features &  96.1$\pm$0.08 &  95.7$\pm$0.06 &  95.9$\pm$0.07 \\
   & no-MW &  89.2$\pm$0.18 &  88.2$\pm$0.11 &  88.7$\pm$0.14 \\
YSO & all-features &  92.8$\pm$0.17 &  95.4$\pm$0.25 &  94.1$\pm$0.20 \\
   & no-MW &  82.9$\pm$0.09 &  89.4$\pm$0.33 &  86.0$\pm$0.15 \\
HMXB & all-features &  91.8$\pm$0.24 &  90.7$\pm$0.66 &  91.2$\pm$0.36 \\
   & no-MW &  92.3$\pm$0.45 &  90.1$\pm$0.37 &  91.2$\pm$0.39 \\
LMXB & all-features &  94.8$\pm$1.33 &  80.7$\pm$0.84 &  87.2$\pm$0.60 \\
   & no-MW &  94.5$\pm$1.02 &  81.7$\pm$0.52 &  87.6$\pm$0.55 \\
ULX & all-features &  71.7$\pm$1.23 &  71.2$\pm$1.10 &  71.5$\pm$1.05 \\
   & no-MW &  61.9$\pm$1.33 &  43.6$\pm$1.34 &  51.2$\pm$1.36 \\
CV & all-features &  60.6$\pm$1.16 &  54.5$\pm$1.50 &  57.4$\pm

In [86]:
print(score_compact.T.replace({'_' , ' '}).to_latex(escape=False , multicolumn=False , column_format = 'lccccccccr' , multirow=True))

\begin{tabular}{lccccccccr}
\toprule
{} &            AGN &                &           STAR &                &            YSO &                &           HMXB &                &           LMXB &                &            ULX &                &             CV &                \\
{} &            all &          no-MW &            all &          no-MW &            all &          no-MW &            all &          no-MW &            all &          no-MW &            all &          no-MW &            all &          no-MW \\
\midrule
precision &  96.8$\pm$0.14 &  91.1$\pm$0.12 &  96.1$\pm$0.08 &  89.2$\pm$0.18 &  92.8$\pm$0.17 &  82.9$\pm$0.09 &  91.8$\pm$0.24 &  92.3$\pm$0.45 &  94.8$\pm$1.33 &  94.5$\pm$1.02 &  71.7$\pm$1.23 &  61.9$\pm$1.33 &  60.6$\pm$1.16 &  56.1$\pm$2.75 \\
recall    &  97.6$\pm$0.15 &  95.1$\pm$0.09 &  95.7$\pm$0.06 &  88.2$\pm$0.11 &  95.4$\pm$0.25 &  89.4$\pm$0.33 &  90.7$\pm$0.66 &  90.1$\pm$0.37 &  80.7$\pm$0.84 &  81.7$\pm$0.52 &  71.2$\pm$1.10 &  43.6$\pm$1.34 &

In [26]:
final_df = pd.concat(final , keys=['All features' , 'no-MW'])
final_df

AGN           STAR            YSO  \
All features precision  96.8$\pm$0.14  96.1$\pm$0.08  92.8$\pm$0.17   
             recall     97.6$\pm$0.15  95.7$\pm$0.06  95.4$\pm$0.25   
             f1_score   97.2$\pm$0.11  95.9$\pm$0.07  94.1$\pm$0.20   
no-MW        precision  91.1$\pm$0.12  89.2$\pm$0.18  82.9$\pm$0.09   
             recall     95.1$\pm$0.09  88.2$\pm$0.11  89.4$\pm$0.33   
             f1_score   93.0$\pm$0.07  88.7$\pm$0.14  86.0$\pm$0.15   

                                 HMXB           LMXB            ULX  \
All features precision  91.8$\pm$0.24  94.8$\pm$1.33  71.7$\pm$1.23   
             recall     90.7$\pm$0.66  80.7$\pm$0.84  71.2$\pm$1.10   
             f1_score   91.2$\pm$0.36  87.2$\pm$0.60  71.5$\pm$1.05   
no-MW        precision  92.3$\pm$0.45  94.5$\pm$1.02  61.9$\pm$1.33   
             recall     90.1$\pm$0.37  81.7$\pm$0.52  43.6$\pm$1.34   
             f1_score   91.2$\pm$0.39  87.6$\pm$0.55  51.2$\pm$1.36   

                                   CV         PULSAR  
All features precision  60.6$\pm$1.16  42.1$\pm$1.93  
             recall     54.5$\pm$1.50  45.3$\pm$1.92  
             f1_score   57.4$\pm$1.33  43.7$\pm$1.89  
no-MW        precision  56.1$\pm$2.75  28.2$\pm$1.86  
             recall     45.1$\pm$1.04  18.6$\pm$0.74  
             f1_score   50.0$\pm$1.69  22.4$\pm$0.87

All features                                        no-MW  \
         precision         recall       f1_score      precision   
AGN  96.8$\pm$0.14  97.6$\pm$0.15  97.2$\pm$0.11  91.1$\pm$0.12   

                                   
            recall       f1_score  
AGN  95.1$\pm$0.09  93.0$\pm$0.07

In [27]:
precision , recall , f1_score = {} ,{} , {}
# for m in ['All features' , 'no-IR' , 'no-optical-UV' , 'no-MW']:
for m in ['All features' ,'no-MW']:
    precision[m] = (final_df.loc[m].loc['precision'].to_frame().T)
    recall[m] = (final_df.loc[m].loc['recall'].to_frame().T)
    f1_score[m] = (final_df.loc[m].loc['f1_score'].to_frame().T)
precision = pd.concat(precision).swaplevel(0,1)
recall = pd.concat(recall).swaplevel(0,1)
f1_score  = pd.concat(f1_score).swaplevel(0,1)
score = pd.concat([precision , recall , f1_score])
score

AGN           STAR            YSO  \
precision All features  96.8$\pm$0.14  96.1$\pm$0.08  92.8$\pm$0.17   
          no-MW         91.1$\pm$0.12  89.2$\pm$0.18  82.9$\pm$0.09   
recall    All features  97.6$\pm$0.15  95.7$\pm$0.06  95.4$\pm$0.25   
          no-MW         95.1$\pm$0.09  88.2$\pm$0.11  89.4$\pm$0.33   
f1_score  All features  97.2$\pm$0.11  95.9$\pm$0.07  94.1$\pm$0.20   
          no-MW         93.0$\pm$0.07  88.7$\pm$0.14  86.0$\pm$0.15   

                                 HMXB           LMXB            ULX  \
precision All features  91.8$\pm$0.24  94.8$\pm$1.33  71.7$\pm$1.23   
          no-MW         92.3$\pm$0.45  94.5$\pm$1.02  61.9$\pm$1.33   
recall    All features  90.7$\pm$0.66  80.7$\pm$0.84  71.2$\pm$1.10   
          no-MW         90.1$\pm$0.37  81.7$\pm$0.52  43.6$\pm$1.34   
f1_score  All features  91.2$\pm$0.36  87.2$\pm$0.60  71.5$\pm$1.05   
          no-MW         91.2$\pm$0.39  87.6$\pm$0.55  51.2$\pm$1.36   

                                   CV         PULSAR  
precision All features  60.6$\pm$1.16  42.1$\pm$1.93  
          no-MW         56.1$\pm$2.75  28.2$\pm$1.86  
recall    All features  54.5$\pm$1.50  45.3$\pm$1.92  
          no-MW         45.1$\pm$1.04  18.6$\pm$0.74  
f1_score  All features  57.4$\pm$1.33  43.7$\pm$1.89  
          no-MW         50.0$\pm$1.69  22.4$\pm$0.87

In [35]:
score.loc['precision']

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
All features,96.8$\pm$0.14,96.1$\pm$0.08,92.8$\pm$0.17,91.8$\pm$0.24,94.8$\pm$1.33,71.7$\pm$1.23,60.6$\pm$1.16,42.1$\pm$1.93
no-MW,91.1$\pm$0.12,89.2$\pm$0.18,82.9$\pm$0.09,92.3$\pm$0.45,94.5$\pm$1.02,61.9$\pm$1.33,56.1$\pm$2.75,28.2$\pm$1.86


In [20]:
print(score.T.replace({'_' , ' '}).to_latex(escape=False , multicolumn=True , column_format = 'lccccccccl' , multirow=True))

\begin{tabular}{lccccccccl}
\toprule
{} & \multicolumn{2}{l}{precision} & \multicolumn{2}{l}{recall} & \multicolumn{2}{l}{f1_score} \\
{} &   All features &          no-MW &   All features &          no-MW &   All features &          no-MW \\
\midrule
AGN    &  96.8$\pm$0.14 &  91.1$\pm$0.12 &  97.6$\pm$0.15 &  95.1$\pm$0.09 &  97.2$\pm$0.11 &  93.0$\pm$0.07 \\
STAR   &  96.1$\pm$0.08 &  89.2$\pm$0.18 &  95.7$\pm$0.06 &  88.2$\pm$0.11 &  95.9$\pm$0.07 &  88.7$\pm$0.14 \\
YSO    &  92.8$\pm$0.17 &  82.9$\pm$0.09 &  95.4$\pm$0.25 &  89.4$\pm$0.33 &  94.1$\pm$0.20 &  86.0$\pm$0.15 \\
HMXB   &  91.8$\pm$0.24 &  92.3$\pm$0.45 &  90.7$\pm$0.66 &  90.1$\pm$0.37 &  91.2$\pm$0.36 &  91.2$\pm$0.39 \\
LMXB   &  94.8$\pm$1.33 &  94.5$\pm$1.02 &  80.7$\pm$0.84 &  81.7$\pm$0.52 &  87.2$\pm$0.60 &  87.6$\pm$0.55 \\
ULX    &  71.7$\pm$1.23 &  61.9$\pm$1.33 &  71.2$\pm$1.10 &  43.6$\pm$1.34 &  71.5$\pm$1.05 &  51.2$\pm$1.36 \\
CV     &  60.6$\pm$1.16 &  56.1$\pm$2.75 &  54.5$\pm$1.50 &  45.1$\pm$1.04 &

In [20]:
final_df.loc['no-MW']

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,91.1$\pm$0.12,89.2$\pm$0.18,82.9$\pm$0.09,92.3$\pm$0.45,94.5$\pm$1.02,61.9$\pm$1.33,56.1$\pm$2.75,28.2$\pm$1.86
recall,95.1$\pm$0.09,88.2$\pm$0.11,89.4$\pm$0.33,90.1$\pm$0.37,81.7$\pm$0.52,43.6$\pm$1.34,45.1$\pm$1.04,18.6$\pm$0.74
f1_score,93.0$\pm$0.07,88.7$\pm$0.14,86.0$\pm$0.15,91.2$\pm$0.39,87.6$\pm$0.55,51.2$\pm$1.36,50.0$\pm$1.69,22.4$\pm$0.87


In [66]:
print(final_df.replace({'_' , ' '}).to_latex(escape=False , multicolumn=True , column_format = 'lccccccccl'))

\begin{tabular}{lccccccccl}
\toprule
                            &          &            AGN &           STAR &            YSO &           HMXB &           LMXB &            ULX &             CV &         PULSAR \\
\midrule
All 41 features & precision &  96.8$\pm$0.14 &  96.1$\pm$0.08 &  92.8$\pm$0.17 &  91.8$\pm$0.24 &  94.8$\pm$1.33 &  71.7$\pm$1.23 &  60.6$\pm$1.16 &  42.1$\pm$1.93 \\
                            & recall &  97.6$\pm$0.15 &  95.7$\pm$0.06 &  95.4$\pm$0.25 &  90.7$\pm$0.66 &  80.7$\pm$0.84 &  71.2$\pm$1.10 &  54.5$\pm$1.50 &  45.3$\pm$1.92 \\
                            & f1_score &  97.2$\pm$0.11 &  95.9$\pm$0.07 &  94.1$\pm$0.20 &  91.2$\pm$0.36 &  87.2$\pm$0.60 &  71.5$\pm$1.05 &  57.4$\pm$1.33 &  43.7$\pm$1.89 \\
color features removed & precision &  96.6$\pm$0.12 &  96.0$\pm$0.17 &  91.8$\pm$0.26 &  91.9$\pm$0.47 &  95.0$\pm$0.61 &  71.6$\pm$0.90 &  61.8$\pm$1.38 &  40.4$\pm$1.35 \\
                            & recall &  97.6$\pm$0.10 &  95.2$\pm$0.14 &  95.3$\p

In [ ]:
mean = mean.reset_index()
display(mean)
print(mean.to_latex(escape=False))